In [30]:
from importlib import reload
import utils
from utils import bandits, snowden, web
reload(utils)
reload(utils.web)
reload(utils.bandits)
reload(utils.snowden)
from utils.bandits import exact_gittins

from templates import neural as neural_agent
reload(neural_agent)

neural_agent_class = neural_agent.NeuralAgent

In [167]:
# datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226/', 1/10)

In [98]:
datasets[0].X.shape, datasets[1].X.shape

((25987, 36), (1999, 36))

In [99]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.31915957978989495
mean after resampling: 0.5
mean before resampling: 0.3166583291645823
mean after resampling: 0.5


In [100]:
train.X.shape, val.X.shape

((16588, 36), (1266, 36))

In [199]:
!pip install tensorboardX

     |████████████████████████████████| 308 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 173 kB 7.5 MB/s eta 0:00:01


In [3]:
def fix_dataset(train, val,H=72):
    train.X = train.X.reshape(-1, 100, H)
    val.X = val.X.reshape(-1, 100, H)

    train.actions = train.actions.reshape(-1)
    val.actions = val.actions.reshape(-1)

In [4]:
import torch
from torch import nn
from torch import optim

from tensorboardX import SummaryWriter

import numpy as np

In [23]:
def learn(train, val, model_ff, epochs=5, batch_size=64, shuffle=True, freq=10,lr=1e-3,criterion = nn.MSELoss()): 
    writer = SummaryWriter()
    
    np.random.seed(1)
    ids_nn = np.arange(train.y.shape[0])
    
    reshape_to_last = lambda x: torch.reshape(x, [np.prod(x.shape[:-1]), x.shape[-1]])

    optimizer = optim.Adam(model_ff.parameters(), lr=lr)

    time_for_print_loss = lambda i: (i + 1) % freq == 0
    
    n_iter = 0
    

    for epoch in np.arange(epochs):
        np.random.shuffle(ids_nn)

        model_ff.train(True)

        for b in np.arange(0, train.y.shape[0], batch_size):
            X_batch = torch.FloatTensor(train.X[ids_nn[b:b+batch_size]])
            X_batch = reshape_to_last(X_batch)
            y_batch = torch.FloatTensor(train.y[ids_nn[b:b+batch_size]])
            a_batch = torch.LongTensor(train.actions[ids_nn[b:b+batch_size]])

            optimizer.zero_grad()
            y_pred_logits = model_ff(X_batch)

            loss = criterion(y_pred_logits, y_batch, a_batch, X_batch)
            loss.backward()

            optimizer.step()

            if (b // batch_size + 1) % freq == 0:
                print('train loss in %d epoch in %d batch: %.5f' %
                  (epoch + 1, b // batch_size + 1, loss.item()))
                
                writer.add_scalar('data/train_loss', loss.item(), n_iter)
                writer.add_scalar('data/epoch', epoch + 1, n_iter)
                writer.add_scalar('data/batch', b // batch_size + 1, n_iter)

                val_loss = 0
                its = 0
                model_ff.train(False)
                for b in np.arange(0, val.y.shape[0], batch_size):
                    its += 1
                    X_batch = torch.FloatTensor(val.X[b:b+batch_size])
                    X_batch = reshape_to_last(X_batch)

                    y_batch = torch.FloatTensor(val.y[b:b+batch_size])
                    a_batch = torch.LongTensor(val.actions[b:b+batch_size])
                    with torch.no_grad():
                        y_pred_logits = model_ff(X_batch)
                    loss = criterion(y_pred_logits, y_batch, a_batch, X_batch)
                    val_loss += loss.item()
                val_loss /= its
                print('val loss in %d epoch: %.5f' % (epoch + 1, val_loss))
                
                writer.add_scalar('data/val_loss', val_loss, n_iter)
                n_iter += 1

In [6]:
class NNWithCusomFeatures(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        INPUT_F_C = INPUT_F + 2 * INPUT_F
        self.model_ff =  nn.Sequential(
            nn.BatchNorm1d(INPUT_F_C),
            nn.Dropout(DROP_P),
            nn.Linear(INPUT_F_C, H),
            nn.Sigmoid(),
            nn.Dropout(DROP_P),
            nn.Linear(H, 1)
#             nn.Sigmoid()
        )

    def forward(self, x):
        lg = torch.log(1 + torch.abs(x))
        sn = torch.sin(x)
        input_x = torch.cat([x, lg, sn], axis=1)
        return self.model_ff(input_x)

In [155]:
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=40, batch_size=32,lr=1e-5, epochs=7)

train loss in 1 epoch in 40 batch: 0.695
train loss in 1 epoch in 80 batch: 0.680
train loss in 1 epoch in 120 batch: 0.696
train loss in 1 epoch in 160 batch: 0.661
train loss in 1 epoch in 200 batch: 0.702
train loss in 1 epoch in 240 batch: 0.665
train loss in 1 epoch in 280 batch: 0.645
train loss in 1 epoch in 320 batch: 0.680
train loss in 1 epoch in 360 batch: 0.683
train loss in 1 epoch in 400 batch: 0.670
train loss in 1 epoch in 440 batch: 0.689
train loss in 1 epoch in 480 batch: 0.630
val loss in 1 epoch: 0.657
train loss in 2 epoch in 40 batch: 0.648
train loss in 2 epoch in 80 batch: 0.672
train loss in 2 epoch in 120 batch: 0.593
train loss in 2 epoch in 160 batch: 0.685
train loss in 2 epoch in 200 batch: 0.696
train loss in 2 epoch in 240 batch: 0.631
train loss in 2 epoch in 280 batch: 0.621
train loss in 2 epoch in 320 batch: 0.661
train loss in 2 epoch in 360 batch: 0.696
train loss in 2 epoch in 400 batch: 0.569
train loss in 2 epoch in 440 batch: 0.673
train loss 

In [156]:
torch.save(model.state_dict(), 'models/nagiss_v1')

In [174]:
# mse loss
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=40, batch_size=32,lr=1e-5, epochs=5)

train loss in 1 epoch in 40 batch: 0.246
train loss in 1 epoch in 80 batch: 0.244
train loss in 1 epoch in 120 batch: 0.248
train loss in 1 epoch in 160 batch: 0.233
train loss in 1 epoch in 200 batch: 0.242
train loss in 1 epoch in 240 batch: 0.238
train loss in 1 epoch in 280 batch: 0.239
train loss in 1 epoch in 320 batch: 0.235
train loss in 1 epoch in 360 batch: 0.245
train loss in 1 epoch in 400 batch: 0.240
train loss in 1 epoch in 440 batch: 0.248
train loss in 1 epoch in 480 batch: 0.216
val loss in 1 epoch: 0.234
train loss in 2 epoch in 40 batch: 0.234
train loss in 2 epoch in 80 batch: 0.233
train loss in 2 epoch in 120 batch: 0.199
train loss in 2 epoch in 160 batch: 0.247
train loss in 2 epoch in 200 batch: 0.256
train loss in 2 epoch in 240 batch: 0.219
train loss in 2 epoch in 280 batch: 0.221
train loss in 2 epoch in 320 batch: 0.227
train loss in 2 epoch in 360 batch: 0.239
train loss in 2 epoch in 400 batch: 0.194
train loss in 2 epoch in 440 batch: 0.240
train loss 

In [175]:
torch.save(model.state_dict(), 'models/nagiss_v2')

In [178]:
# v1
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

632.0 611.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
676.0 642.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
599.0 543.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
624.0 621.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
606.0 639.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
607.0 591.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
609.0 552.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
626.0 606.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
630.0 603.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py
615.0 615.0 tmp/b_0.9817500612110714.py tmp/b_0.6737446183327064.py


(0.012154502623372141,
 20.1,
 25.34738645304482,
 0.9,
 'tmp/b_0.9817500612110714.py',
 'tmp/b_0.6737446183327064.py',
 array([632., 676., 599., 624., 606., 607., 609., 626., 630., 615.]),
 array([611., 642., 543., 621., 639., 591., 552., 606., 603., 615.]))

In [180]:
# v2
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text= utils.bandits.neural),
    verbose=True)

678.0 661.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
700.0 695.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
664.0 660.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
597.0 615.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
659.0 696.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
601.0 581.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
626.0 623.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
536.0 607.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
580.0 550.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py
636.0 562.0 tmp/b_0.054213267519393926.py tmp/b_0.236142134925876.py


(0.8180618145466889,
 2.7,
 37.11616898334202,
 0.7,
 'tmp/b_0.054213267519393926.py',
 'tmp/b_0.236142134925876.py',
 array([678., 700., 664., 597., 659., 601., 626., 536., 580., 636.]),
 array([661., 695., 660., 615., 696., 581., 623., 607., 550., 562.]))

In [6]:
# v2
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text= utils.bandits.neural),
    verbose=True)

657.0 685.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
717.0 717.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
691.0 721.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
629.0 671.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
624.0 662.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
677.0 627.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
579.0 572.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
594.0 621.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
617.0 593.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py
629.0 634.0 tmp/b_0.4681855452475341.py tmp/b_0.815911715993058.py


(0.31907279631279994,
 -8.9,
 28.246946737656444,
 0.4,
 'tmp/b_0.4681855452475341.py',
 'tmp/b_0.815911715993058.py',
 array([657., 717., 691., 629., 624., 677., 579., 594., 617., 629.]),
 array([685., 717., 721., 671., 662., 627., 572., 621., 593., 634.]))

### all nagiss games

In [7]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226_v2/', 1/10)

sessions/nagiss/20201226_v2/ ['sessions/nagiss/20201226_v2/7994638.json', 'sessions/nagiss/20201226_v2/7991093.json', 'sessions/nagiss/20201226_v2/7988820.json', 'sessions/nagiss/20201226_v2/8386716.json', 'sessions/nagiss/20201226_v2/8038792.json', 'sessions/nagiss/20201226_v2/8233786.json', 'sessions/nagiss/20201226_v2/8073493.json', 'sessions/nagiss/20201226_v2/8012231.json', 'sessions/nagiss/20201226_v2/7995268.json', 'sessions/nagiss/20201226_v2/8046220.json', 'sessions/nagiss/20201226_v2/8321309.json', 'sessions/nagiss/20201226_v2/8102725.json', 'sessions/nagiss/20201226_v2/8146617.json', 'sessions/nagiss/20201226_v2/7986639.json', 'sessions/nagiss/20201226_v2/8029737.json', 'sessions/nagiss/20201226_v2/8089900.json', 'sessions/nagiss/20201226_v2/8354985.json', 'sessions/nagiss/20201226_v2/7995644.json', 'sessions/nagiss/20201226_v2/7998867.json', 'sessions/nagiss/20201226_v2/8257767.json', 'sessions/nagiss/20201226_v2/7992085.json', 'sessions/nagiss/20201226_v2/7987842.json', 's

In [11]:
datasets[0].X.shape, datasets[1].X.shape

((133933, 36), (13993, 36))

In [12]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.31643433657127074
mean after resampling: 0.5
mean before resampling: 0.3148002572714929
mean after resampling: 0.5


In [13]:
train.X.shape, val.X.shape

((84762, 36), (8810, 36))

In [ ]:
# class NNWithCusomFeatures(nn.Module):
#     def __init__(self, INPUT_F, DROP_P, H):
#         super().__init__()
#         INPUT_F_C = INPUT_F + 2 * INPUT_F
#         self.model_ff_1 =  nn.Sequential(
#             nn.BatchNorm1d(INPUT_F_C),
#             nn.Dropout(DROP_P),
#             nn.Linear(INPUT_F_C, H),
#             nn.Sigmoid(),
#             nn.Dropout(DROP_P)
#         )
#         self.model_ff_2 =  nn.Sequential(
#             nn.Linear(H, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         lg = torch.log(1 + torch.abs(x))
#         sn = torch.sin(x)
#         input_x = torch.cat([x, lg, sn], axis=1)
#         out1 =  self.model_ff_1(input_x)

In [76]:
# mse loss
INPUT_F = 36
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=500, batch_size=32,lr=1e-5, epochs=5)

train loss in 1 epoch in 500 batch: 0.24326
train loss in 1 epoch in 1000 batch: 0.23294
train loss in 1 epoch in 1500 batch: 0.25006
train loss in 1 epoch in 2000 batch: 0.20686
train loss in 1 epoch in 2500 batch: 0.21678
val loss in 1 epoch: 0.22331
train loss in 2 epoch in 500 batch: 0.26177
train loss in 2 epoch in 1000 batch: 0.19144
train loss in 2 epoch in 1500 batch: 0.17022
train loss in 2 epoch in 2000 batch: 0.23671
train loss in 2 epoch in 2500 batch: 0.22735
val loss in 2 epoch: 0.22240
train loss in 3 epoch in 500 batch: 0.18364
train loss in 3 epoch in 1000 batch: 0.25060
train loss in 3 epoch in 1500 batch: 0.25504
train loss in 3 epoch in 2000 batch: 0.21704
train loss in 3 epoch in 2500 batch: 0.20117
val loss in 3 epoch: 0.22189
train loss in 4 epoch in 500 batch: 0.17385
train loss in 4 epoch in 1000 batch: 0.22882
train loss in 4 epoch in 1500 batch: 0.22496
train loss in 4 epoch in 2000 batch: 0.20567
train loss in 4 epoch in 2500 batch: 0.22413
val loss in 4 epo

In [77]:
torch.save(model.state_dict(), 'models/nagiss_v3')

In [79]:
# v3
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

619.0 538.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
552.0 574.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
641.0 652.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
631.0 601.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
703.0 651.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
643.0 666.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
617.0 628.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
607.0 574.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
638.0 660.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py
618.0 632.0 tmp/b_0.5394941534468733.py tmp/b_0.9677390896108925.py


(0.4021809806045401,
 9.3,
 35.105697543276364,
 0.4,
 'tmp/b_0.5394941534468733.py',
 'tmp/b_0.9677390896108925.py',
 array([619., 552., 641., 631., 703., 643., 617., 607., 638., 618.]),
 array([538., 574., 652., 601., 651., 666., 628., 574., 660., 632.]))

In [116]:
#with mean features

datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226_v2/', 1/10)

sessions/nagiss/20201226_v2/ ['sessions/nagiss/20201226_v2/7994638.json', 'sessions/nagiss/20201226_v2/7991093.json', 'sessions/nagiss/20201226_v2/7988820.json', 'sessions/nagiss/20201226_v2/8386716.json', 'sessions/nagiss/20201226_v2/8038792.json', 'sessions/nagiss/20201226_v2/8233786.json', 'sessions/nagiss/20201226_v2/8073493.json', 'sessions/nagiss/20201226_v2/8012231.json', 'sessions/nagiss/20201226_v2/7995268.json', 'sessions/nagiss/20201226_v2/8046220.json', 'sessions/nagiss/20201226_v2/8321309.json', 'sessions/nagiss/20201226_v2/8102725.json', 'sessions/nagiss/20201226_v2/8146617.json', 'sessions/nagiss/20201226_v2/7986639.json', 'sessions/nagiss/20201226_v2/8029737.json', 'sessions/nagiss/20201226_v2/8089900.json', 'sessions/nagiss/20201226_v2/8354985.json', 'sessions/nagiss/20201226_v2/7995644.json', 'sessions/nagiss/20201226_v2/7998867.json', 'sessions/nagiss/20201226_v2/8257767.json', 'sessions/nagiss/20201226_v2/7992085.json', 'sessions/nagiss/20201226_v2/7987842.json', 's

  0%|          | 0/67 [00:00<?, ?it/s]

error while parse session sessions/nagiss/20201226_v2/.ipynb_checkpoints/8002300-checkpoint.json : 


100%|██████████| 7/7 [01:19<00:00, 11.38s/it]


In [117]:
datasets[0].X.shape, datasets[1].X.shape

((133933, 72), (13993, 72))

In [118]:
train = snowden.resample_eq(datasets[0])
val = snowden.resample_eq(datasets[1])

mean before resampling: 0.3157548923715589
mean after resampling: 0.5
mean before resampling: 0.3213035088973058
mean after resampling: 0.5


In [217]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train, val, model, freq=50, batch_size=512,lr=1e-4, epochs=10)

train loss in 1 epoch in 50 batch: 0.22536
train loss in 1 epoch in 100 batch: 0.22169
train loss in 1 epoch in 150 batch: 0.21847
val loss in 1 epoch: 0.22192
train loss in 2 epoch in 50 batch: 0.21630
train loss in 2 epoch in 100 batch: 0.22580
train loss in 2 epoch in 150 batch: 0.22800
val loss in 2 epoch: 0.22128
train loss in 3 epoch in 50 batch: 0.21413
train loss in 3 epoch in 100 batch: 0.22134
train loss in 3 epoch in 150 batch: 0.22260
val loss in 3 epoch: 0.22139
train loss in 4 epoch in 50 batch: 0.21507
train loss in 4 epoch in 100 batch: 0.21997
train loss in 4 epoch in 150 batch: 0.22291
val loss in 4 epoch: 0.22118
train loss in 5 epoch in 50 batch: 0.22766
train loss in 5 epoch in 100 batch: 0.22133
train loss in 5 epoch in 150 batch: 0.21585
val loss in 5 epoch: 0.22130
train loss in 6 epoch in 50 batch: 0.22414
train loss in 6 epoch in 100 batch: 0.21673
train loss in 6 epoch in 150 batch: 0.21959
val loss in 6 epoch: 0.22122
train loss in 7 epoch in 50 batch: 0.214

In [218]:
torch.save(model.state_dict(), 'models/nagiss_v4')

In [220]:
# v4
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

364.0 742.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
395.0 686.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
323.0 666.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
390.0 753.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
394.0 618.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
366.0 663.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
374.0 620.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
398.0 731.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
380.0 682.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py
386.0 745.0 tmp/b_0.624265024014975.py tmp/b_0.8223440142039435.py


(2.2900685246041295e-93,
 -313.6,
 48.3822281421598,
 0.0,
 'tmp/b_0.624265024014975.py',
 'tmp/b_0.8223440142039435.py',
 array([364., 395., 323., 390., 394., 366., 374., 398., 380., 386.]),
 array([742., 686., 666., 753., 618., 663., 620., 731., 682., 745.]))

In [127]:
# # v4 vs default + mean features
# utils.bandits.compare(
#     utils.bandits.Agent(text=neural_with_nagiss),
#     utils.bandits.Agent(text=utils.bandits.neural),
#     verbose=True)

In [139]:
# v3 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

637.0 624.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
652.0 693.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
600.0 628.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
597.0 611.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
646.0 634.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
624.0 627.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
595.0 599.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
605.0 596.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
664.0 672.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py
628.0 608.0 tmp/b_0.00783945348032078.py tmp/b_0.08316848165655322.py


(0.44715964885505466,
 -4.4,
 18.30409790183608,
 0.4,
 'tmp/b_0.00783945348032078.py',
 'tmp/b_0.08316848165655322.py',
 array([637., 652., 600., 597., 646., 624., 595., 605., 664., 628.]),
 array([624., 693., 628., 611., 634., 627., 599., 596., 672., 608.]))

In [141]:
# v2 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

617.0 640.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
644.0 619.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
679.0 694.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
645.0 617.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
601.0 629.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
628.0 617.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
639.0 634.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
651.0 658.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
736.0 647.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py
586.0 588.0 tmp/b_0.2329175121645335.py tmp/b_0.9947346263005474.py


(0.41522101422787383,
 8.3,
 32.21505859066533,
 0.5,
 'tmp/b_0.2329175121645335.py',
 'tmp/b_0.9947346263005474.py',
 array([617., 644., 679., 645., 601., 628., 639., 651., 736., 586.]),
 array([640., 619., 694., 617., 629., 617., 634., 658., 647., 588.]))

In [137]:
# v4 as feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

627.0 664.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
563.0 601.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
579.0 583.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
673.0 630.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
648.0 693.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
633.0 632.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
625.0 616.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
607.0 577.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
625.0 655.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py
617.0 651.0 tmp/b_0.6232887914692327.py tmp/b_0.7812314428145001.py


(0.25859498040269646,
 -10.5,
 29.391325250828686,
 0.4,
 'tmp/b_0.6232887914692327.py',
 'tmp/b_0.7812314428145001.py',
 array([627., 563., 579., 673., 648., 633., 625., 607., 625., 617.]),
 array([664., 601., 583., 630., 693., 632., 616., 577., 655., 651.]))

In [142]:
# v2 as feature vs only v2
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
neural_with_only_nagiss = utils.bandits.neural.format("use_only_nagiss=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_with_only_nagiss),
    verbose=True)

538.0 550.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
633.0 611.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
589.0 594.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
614.0 638.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
631.0 664.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
656.0 644.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
529.0 583.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
610.0 635.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
641.0 625.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py
664.0 632.0 tmp/b_0.03981686451690292.py tmp/b_0.8435219654502392.py


(0.38850022498604,
 -7.1,
 26.036320784627,
 0.4,
 'tmp/b_0.03981686451690292.py',
 'tmp/b_0.8435219654502392.py',
 array([538., 633., 589., 614., 631., 656., 529., 610., 641., 664.]),
 array([550., 611., 594., 638., 664., 644., 583., 635., 625., 632.]))

In [161]:
save_dataset(datasets[0], 'test_save_nagiss_train.npy')
save_dataset(datasets[1], 'test_save_nagiss_val.npy')

fit on Temporal Difference

In [173]:
datasets[0].X[2]

array([ 1.54513832e+00,  5.82159002e-01,  2.12729732e+00,  2.00000000e+00,
        1.99800000e+03,  1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        7.26338676e-01,  8.22673418e-01,  1.55853035e+00,  7.48466609e-01,
        4.76706051e-01,  9.21358662e-01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.44497329e+00,  1.08565598e+00,  2.28020948e+00,
        2.72875138e+00,  6.66666667e-01,  6.66666667e-01,  6.66666667e-01,
        9.90000000e-01, -1.00000000e-02,  9.80000000e-01,  0.00000000e+00,
        0.00000000e+00,  9.90000000e-01, -1.00000000e-02, -2.00000000e-02,
        2.42604406e-01,  5.24751214e-02, -1.16985524e-01,  3.17383712e-01,
       -2.61674838e-02,  1.74026792e-01, -1.00000000e-02, -1.00000000e-02,
        0.00000000e+00,  

In [7]:
class NNWithCusomFeatures2(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        INPUT_F_C = INPUT_F + 2 * INPUT_F
        self.model_ff =  nn.Sequential(
            nn.BatchNorm1d(INPUT_F_C),
            nn.Dropout(DROP_P),
            nn.Linear(INPUT_F_C, H),
            nn.Sigmoid(),
            nn.Linear(H, H),
            nn.Sigmoid(),
            nn.Dropout(DROP_P),
            nn.Linear(H, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        lg = torch.log(1 + torch.abs(x))
        sn = torch.sin(x)
        input_x = torch.cat([x, lg, sn], axis=1)
        return self.model_ff(input_x)

class TDModel(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        self.model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

    def forward(self, x):
        left = x[:, : INPUT_F]
        right = x[:, INPUT_F :]
        return self.model(left) - self.model(right).detach()
    
class DoubleTDModel(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        self.modelA = NNWithCusomFeatures2(INPUT_F, DROP_P, H)
        self.modelB = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

    def forward(self, x):
        left = x[:, : INPUT_F]
        right = x[:, INPUT_F :]
        ab = self.modelA(left) - self.modelB(right)
        ba = self.modelB(left) - self.modelA(right)
        return (ab + ba) / 2

In [8]:
def make_td_dataset(d : utils.snowden.Dataset):
    left = np.concatenate([d.X, np.zeros((1, d.X.shape[1]))], axis=0)
    right = np.concatenate([np.zeros((1, d.X.shape[1])), d.X], axis=0)
    y = np.concatenate([d.y, [0]])
    X = np.concatenate([left, right], axis=1)
    return utils.snowden.Dataset(X, y, d.sessions)

In [181]:
td_train = make_td_dataset(datasets[0])
td_val = make_td_dataset(datasets[1])

In [221]:
td_train.save_dataset('datasets/td_train_nagiss_20201226')
td_val.save_dataset('datasets/td_val_nagiss_20201226')

In [184]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.05

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(datasets[0], datasets[1], model, freq=500, batch_size=32,lr=1e-5, epochs=1)

train loss in 1 epoch in 500 batch: 0.16636
train loss in 1 epoch in 1000 batch: 0.16476
train loss in 1 epoch in 1500 batch: 0.17953
train loss in 1 epoch in 2000 batch: 0.30793
train loss in 1 epoch in 2500 batch: 0.22230
train loss in 1 epoch in 3000 batch: 0.20506
train loss in 1 epoch in 3500 batch: 0.14479
train loss in 1 epoch in 4000 batch: 0.18252
val loss in 1 epoch: 0.19520


In [206]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.1

tdmodel = TDModel(INPUT_F, DROP_P, H)

learn(td_train, td_val, tdmodel, freq=50, batch_size=512,lr=1e-5, epochs=10)

train loss in 1 epoch in 50 batch: 0.33739
train loss in 1 epoch in 100 batch: 0.29898
train loss in 1 epoch in 150 batch: 0.30539
train loss in 1 epoch in 200 batch: 0.33259
train loss in 1 epoch in 250 batch: 0.33338
val loss in 1 epoch: 0.31717
train loss in 2 epoch in 50 batch: 0.30092
train loss in 2 epoch in 100 batch: 0.32005
train loss in 2 epoch in 150 batch: 0.31823
train loss in 2 epoch in 200 batch: 0.31090
train loss in 2 epoch in 250 batch: 0.32401
val loss in 2 epoch: 0.31707
train loss in 3 epoch in 50 batch: 0.30618
train loss in 3 epoch in 100 batch: 0.33236
train loss in 3 epoch in 150 batch: 0.33858
train loss in 3 epoch in 200 batch: 0.31367
train loss in 3 epoch in 250 batch: 0.31520
val loss in 3 epoch: 0.31703
train loss in 4 epoch in 50 batch: 0.30974
train loss in 4 epoch in 100 batch: 0.33966
train loss in 4 epoch in 150 batch: 0.32230
train loss in 4 epoch in 200 batch: 0.32780
train loss in 4 epoch in 250 batch: 0.31112
val loss in 4 epoch: 0.31701
train lo

In [207]:
torch.save(tdmodel.model.state_dict(), 'models/nagiss_v5')

In [250]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.01

tdmodel = TDModel(INPUT_F, DROP_P, H)

learn(td_train, td_val, tdmodel, freq=50, batch_size=512,lr=1e-5, epochs=10)

train loss in 1 epoch in 50 batch: 0.33434
train loss in 1 epoch in 100 batch: 0.29501
train loss in 1 epoch in 150 batch: 0.30367
train loss in 1 epoch in 200 batch: 0.33173
train loss in 1 epoch in 250 batch: 0.32840
val loss in 1 epoch: 0.31808
train loss in 2 epoch in 50 batch: 0.30069
train loss in 2 epoch in 100 batch: 0.32157
train loss in 2 epoch in 150 batch: 0.32148
train loss in 2 epoch in 200 batch: 0.31430
train loss in 2 epoch in 250 batch: 0.32340
val loss in 2 epoch: 0.31920
train loss in 3 epoch in 50 batch: 0.30729
train loss in 3 epoch in 100 batch: 0.33402
train loss in 3 epoch in 150 batch: 0.33552
train loss in 3 epoch in 200 batch: 0.31429
train loss in 3 epoch in 250 batch: 0.31118
val loss in 3 epoch: 0.31922
train loss in 4 epoch in 50 batch: 0.30944
train loss in 4 epoch in 100 batch: 0.34393
train loss in 4 epoch in 150 batch: 0.32317
train loss in 4 epoch in 200 batch: 0.32753
train loss in 4 epoch in 250 batch: 0.31087
val loss in 4 epoch: 0.31874
train lo

In [209]:
# v5 only
# neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss=True", "")
neural_with_only_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_only_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

579.0 653.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
585.0 608.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
589.0 628.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
590.0 651.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
573.0 644.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
573.0 657.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
590.0 641.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
568.0 701.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
595.0 642.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py
566.0 629.0 tmp/b_0.5780403019658866.py tmp/b_0.7839141865156775.py


(6.64840967158245e-13,
 -64.6,
 28.42604439594085,
 0.0,
 'tmp/b_0.5780403019658866.py',
 'tmp/b_0.7839141865156775.py',
 array([579., 585., 589., 590., 573., 573., 590., 568., 595., 566.]),
 array([653., 608., 628., 651., 644., 657., 641., 701., 642., 629.]))

In [212]:
# v5 features
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

674.0 678.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
632.0 639.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
648.0 679.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
611.0 585.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
705.0 660.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
659.0 625.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
586.0 558.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
694.0 668.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
720.0 681.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py
660.0 639.0 tmp/b_0.3813700667284041.py tmp/b_0.7124470702854484.py


(0.013848867420041112,
 17.7,
 22.742251427684113,
 0.7,
 'tmp/b_0.3813700667284041.py',
 'tmp/b_0.7124470702854484.py',
 array([674., 632., 648., 611., 705., 659., 586., 694., 720., 660.]),
 array([678., 639., 679., 585., 660., 625., 558., 668., 681., 639.]))

## top 10 dataset

In [149]:
submissions = [18775092, 18808528, 18645426,
               18841795, 18844737, 18817525,
               18689820, 18818826, 18667443, 18814531]

In [151]:
for sub in submissions:
    utils.web.download_all_sessions_for_submission(sub, 'sessions/20201227/{}'.format(sub), delta=0.1)
    print('{} OK'.format(sub))

In [229]:
datasets_top10 = snowden.collect_dataset_from_dir(
    neural_agent_class,
    'sessions/20201227/',
    val_ratio=2/10,
    n_jobs=8)

sessions/20201227/ ['sessions/20201227/18817525/8314372.json', 'sessions/20201227/18817525/8397408.json', 'sessions/20201227/18817525/8415499.json', 'sessions/20201227/18817525/8318832.json', 'sessions/20201227/18817525/8315742.json', 'sessions/20201227/18817525/8324361.json', 'sessions/20201227/18817525/8317114.json', 'sessions/20201227/18817525/8519185.json', 'sessions/20201227/18817525/8461310.json', 'sessions/20201227/18817525/8641350.json', 'sessions/20201227/18817525/8321949.json', 'sessions/20201227/18817525/8625587.json', 'sessions/20201227/18817525/8314040.json', 'sessions/20201227/18817525/8570033.json', 'sessions/20201227/18817525/8657196.json', 'sessions/20201227/18817525/8318147.json', 'sessions/20201227/18817525/8621966.json', 'sessions/20201227/18817525/8626309.json', 'sessions/20201227/18817525/8323329.json', 'sessions/20201227/18817525/8360828.json', 'sessions/20201227/18817525/8316778.json', 'sessions/20201227/18817525/8343671.json', 'sessions/20201227/18817525/837670

100%|██████████| 191/191 [08:34<00:00,  2.69s/it]


In [230]:
train10 = make_td_dataset(datasets_top10[0])
val10 = make_td_dataset(datasets_top10[1])

In [231]:
train10.save_dataset('datasets/td_train_top_10_20201227')
val10.save_dataset('datasets/td_val_top_10_20201227')

In [232]:
train10.X.shape

(1529236, 144)

In [313]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.01

model = TDModel(INPUT_F, DROP_P, H)

learn(train10, val10, model, freq=500, batch_size=1024,lr=1e-5, epochs=10)

train loss in 1 epoch in 500 batch: 0.31017
val loss in 1 epoch: 0.32012
train loss in 1 epoch in 1000 batch: 0.33459
val loss in 1 epoch: 0.31990
train loss in 2 epoch in 500 batch: 0.29971
val loss in 2 epoch: 0.31826
train loss in 2 epoch in 1000 batch: 0.31261
val loss in 2 epoch: 0.31808
train loss in 3 epoch in 500 batch: 0.30555
val loss in 3 epoch: 0.31801
train loss in 3 epoch in 1000 batch: 0.31830
val loss in 3 epoch: 0.31801
train loss in 4 epoch in 500 batch: 0.32305
val loss in 4 epoch: 0.31802
train loss in 4 epoch in 1000 batch: 0.31833
val loss in 4 epoch: 0.31803
train loss in 5 epoch in 500 batch: 0.29391
val loss in 5 epoch: 0.31804
train loss in 5 epoch in 1000 batch: 0.32989
val loss in 5 epoch: 0.31804
train loss in 6 epoch in 500 batch: 0.32324
val loss in 6 epoch: 0.31805
train loss in 6 epoch in 1000 batch: 0.29496
val loss in 6 epoch: 0.31805
train loss in 7 epoch in 500 batch: 0.30279
val loss in 7 epoch: 0.31806
train loss in 7 epoch in 1000 batch: 0.32812


In [314]:
torch.save(model.model.state_dict(), 'models/nagiss_v7')

In [289]:
train_top10 = snowden.resample_eq(datasets_top10[0])
val_top10 = snowden.resample_eq(datasets_top10[1])

mean before resampling: 0.3173560636527414
mean after resampling: 0.5
mean before resampling: 0.31813026932314326
mean after resampling: 0.5


In [291]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.01

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

learn(train_top10, val_top10, model, freq=500, batch_size=256,lr=1e-5, epochs=10)

train loss in 1 epoch in 500 batch: 0.22432
val loss in 1 epoch: 0.22639
train loss in 1 epoch in 1000 batch: 0.22962
val loss in 1 epoch: 0.22300
train loss in 1 epoch in 1500 batch: 0.23037
val loss in 1 epoch: 0.22201
train loss in 1 epoch in 2000 batch: 0.22566
val loss in 1 epoch: 0.22158
train loss in 1 epoch in 2500 batch: 0.22240
val loss in 1 epoch: 0.22137
train loss in 1 epoch in 3000 batch: 0.22608
val loss in 1 epoch: 0.22123
train loss in 1 epoch in 3500 batch: 0.23233
val loss in 1 epoch: 0.22114
train loss in 2 epoch in 500 batch: 0.21473
val loss in 2 epoch: 0.22107
train loss in 2 epoch in 1000 batch: 0.22262
val loss in 2 epoch: 0.22105
train loss in 2 epoch in 1500 batch: 0.24129
val loss in 2 epoch: 0.22099
train loss in 2 epoch in 2000 batch: 0.23092
val loss in 2 epoch: 0.22098
train loss in 2 epoch in 2500 batch: 0.26149
val loss in 2 epoch: 0.22096
train loss in 2 epoch in 3000 batch: 0.20146
val loss in 2 epoch: 0.22094
train loss in 2 epoch in 3500 batch: 0.2

In [293]:
torch.save(model.state_dict(), 'models/nagiss_v6')

In [295]:
# v6 feature
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

667.0 635.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
677.0 631.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
639.0 649.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
598.0 611.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
580.0 571.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
563.0 533.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
614.0 625.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
643.0 620.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
613.0 616.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py
584.0 604.0 tmp/b_0.5915544147234911.py tmp/b_0.6092418235378909.py


(0.22897157536219315,
 8.3,
 21.817653402692052,
 0.5,
 'tmp/b_0.5915544147234911.py',
 'tmp/b_0.6092418235378909.py',
 array([667., 677., 639., 598., 580., 563., 614., 643., 613., 584.]),
 array([635., 631., 649., 611., 571., 533., 625., 620., 616., 604.]))

In [296]:
# v6 only
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=utils.bandits.neural),
    verbose=True)

601.0 621.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
595.0 565.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
599.0 715.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
521.0 627.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
601.0 633.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
540.0 651.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
487.0 603.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
606.0 609.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
564.0 610.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py
572.0 612.0 tmp/b_0.041844375173799464.py tmp/b_0.928490600352431.py


(0.0004145997283126856,
 -56.0,
 50.157751145760116,
 0.1,
 'tmp/b_0.041844375173799464.py',
 'tmp/b_0.928490600352431.py',
 array([601., 595., 599., 521., 601., 540., 487., 606., 564., 572.]),
 array([621., 565., 715., 627., 633., 651., 603., 609., 610., 612.]))

In [306]:
utils.bandits.bench(
    utils.bandits.Agent(
        text=utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")))

['gittins', (2.2338113477826215e-05, 41.9, 31.2488399784696)]
['gittins_with_random', (0.04874871594922682, 26.2, 42.039980970499975)]
['gittins_with_count_my', (0.040665651939938506, 21.5, 33.21520736048475)]
['gittins_with_count_rival_drift', (0.14417327148436906, 20.7, 44.82198121457819)]
['gittins_with_my_and_count_rival_drift', (0.17334983983393915, 13.4, 31.122981862283055)]
['gittins_bb', (0.3269775255193714, -13.3, 42.9069924371308)]
['gittins_bb_delta', (0.671288128512241, -4.0, 29.80603965641863)]
['softmax_ucb', (7.764515993879431e-16, 60.3, 23.664530420018902)]
['multiarmed_bandit_agent', (1.0, 0.0, 19.662146373170962)]
['upper_confidence', (4.041218931197669e-47, 159.2, 34.91933561796387)]
['ucb_decay', (1.0455956308323003e-06, 57.1, 36.97958896472485)]
['bayesian_ucb', (0.40061517620458387, -8.1, 30.47441549890662)]
['thompson', (0.7564081166054748, -2.8, 28.54400112107621)]
['neural', (0.7840180300583126, -2.9, 33.45878061137316)]
<string>:82: RuntimeWarning: invalid val

[['gittins', (2.2338113477826215e-05, 41.9, 31.2488399784696)],
 ['gittins_with_random', (0.04874871594922682, 26.2, 42.039980970499975)],
 ['gittins_with_count_my', (0.040665651939938506, 21.5, 33.21520736048475)],
 ['gittins_with_count_rival_drift',
  (0.14417327148436906, 20.7, 44.82198121457819)],
 ['gittins_with_my_and_count_rival_drift',
  (0.17334983983393915, 13.4, 31.122981862283055)],
 ['gittins_bb', (0.3269775255193714, -13.3, 42.9069924371308)],
 ['gittins_bb_delta', (0.671288128512241, -4.0, 29.80603965641863)],
 ['softmax_ucb', (7.764515993879431e-16, 60.3, 23.664530420018902)],
 ['multiarmed_bandit_agent', (1.0, 0.0, 19.662146373170962)],
 ['upper_confidence', (4.041218931197669e-47, 159.2, 34.91933561796387)],
 ['ucb_decay', (1.0455956308323003e-06, 57.1, 36.97958896472485)],
 ['bayesian_ucb', (0.40061517620458387, -8.1, 30.47441549890662)],
 ['thompson', (0.7564081166054748, -2.8, 28.54400112107621)],
 ['neural', (0.7840180300583126, -2.9, 33.45878061137316)],
 ['optim

### Policy gradient

In [37]:
datasets = snowden.collect_dataset_from_dir(neural_agent_class, 'sessions/nagiss/20201226/', 1/10)

sessions/nagiss/20201226/ ['sessions/nagiss/20201226/8389416.json', 'sessions/nagiss/20201226/8528396.json', 'sessions/nagiss/20201226/8415103.json', 'sessions/nagiss/20201226/8354920.json', 'sessions/nagiss/20201226/8499335.json', 'sessions/nagiss/20201226/8441958.json', 'sessions/nagiss/20201226/8377070.json', 'sessions/nagiss/20201226/8513521.json', 'sessions/nagiss/20201226/8364900.json', 'sessions/nagiss/20201226/8428032.json', 'sessions/nagiss/20201226/8485987.json', 'sessions/nagiss/20201226/8455991.json', 'sessions/nagiss/20201226/8471153.json', 'sessions/nagiss/20201226/8401973.json', 'sessions/nagiss/20201226/.ipynb_checkpoints/8354920-checkpoint.json']



100%|██████████| 14/14 [02:40<00:00, 11.45s/it]

100%|██████████| 1/1 [00:11<00:00, 11.04s/it]


In [38]:
# datasets[0].save_dataset('datasets/all_actions_train_nagiss_20201226')
# datasets[1].save_dataset('datasets/all_actions_val_nagiss_20201226')

In [7]:
train = snowden.Dataset.load_dataset('datasets/all_actions_train_nagiss_20201226.npy')
val = snowden.Dataset.load_dataset('datasets/all_actions_val_nagiss_20201226.npy')

In [8]:
train.actions.shape

(14, 1999)

In [20]:
val.X.shape, val.actions.shape

((1999, 100, 72), (1999,))

In [44]:
# mse loss
INPUT_F = 72
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.sum(policy * rewards_batch) / torch.sum(rewards_batch)
    return out

learn(train, val, model, freq=200, batch_size=64,lr=1e-4, epochs=10, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 2.44914
val loss in 1 epoch: 3.19189
train loss in 1 epoch in 400 batch: 2.12746
val loss in 1 epoch: 3.11925
train loss in 2 epoch in 200 batch: 2.52760
val loss in 2 epoch: 3.08872
train loss in 2 epoch in 400 batch: 2.57668
val loss in 2 epoch: 3.01324
train loss in 3 epoch in 200 batch: 2.47165
val loss in 3 epoch: 2.98740
train loss in 3 epoch in 400 batch: 3.53690
val loss in 3 epoch: 2.99072
train loss in 4 epoch in 200 batch: 2.38290
val loss in 4 epoch: 2.94655
train loss in 4 epoch in 400 batch: 1.71899
val loss in 4 epoch: 2.95541
train loss in 5 epoch in 200 batch: 3.12862
val loss in 5 epoch: 2.96041
train loss in 5 epoch in 400 batch: 2.82046
val loss in 5 epoch: 2.93376
train loss in 6 epoch in 200 batch: 2.19779
val loss in 6 epoch: 2.91537
train loss in 6 epoch in 400 batch: 3.29692
val loss in 6 epoch: 2.89947
train loss in 7 epoch in 200 batch: 1.82034
val loss in 7 epoch: 2.90066
train loss in 7 epoch in 400 batch: 2.36200
val los

In [30]:
torch.save(model.state_dict(), 'models/nagiss_v7')

In [32]:
# v7
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

657.0 673.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
662.0 651.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
602.0 600.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
556.0 565.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
640.0 594.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
645.0 669.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
622.0 661.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
562.0 509.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
629.0 607.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py
625.0 653.0 tmp/b_0.5539695728453294.py tmp/b_0.3664986483537218.py


(0.847144391522459,
 1.8,
 29.52896882723811,
 0.5,
 'tmp/b_0.5539695728453294.py',
 'tmp/b_0.3664986483537218.py',
 array([657., 662., 602., 556., 640., 645., 622., 562., 629., 625.]),
 array([673., 651., 600., 565., 594., 669., 661., 509., 607., 653.]))

In [40]:
# # v7 + eps greedy - bad
# neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
# neural_default = utils.bandits.neural.format("use_mean=False", "")
# utils.bandits.compare(
#     utils.bandits.Agent(text=neural_with_nagiss),
#     utils.bandits.Agent(text=neural_default),
#     verbose=True)

In [ ]:
utils.bandits.compare()

In [36]:
# v7
neural_with_nagiss = utils.bandits.neural.format("use_feature_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

607.0 641.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
678.0 689.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
622.0 642.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
652.0 639.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
621.0 587.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
584.0 584.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
666.0 689.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
597.0 624.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
636.0 607.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py
566.0 620.0 tmp/b_0.7818590056214438.py tmp/b_0.7584076114252468.py


(0.2714717561979586,
 -9.3,
 26.74341040331244,
 0.4,
 'tmp/b_0.7818590056214438.py',
 'tmp/b_0.7584076114252468.py',
 array([607., 678., 622., 652., 621., 584., 666., 597., 636., 566.]),
 array([641., 689., 642., 639., 587., 584., 689., 624., 607., 620.]))

## nagiss_big

In [8]:
datasets_nagiss_big = snowden.collect_dataset_from_dir(
    neural_agent_class,
    'sessions/nagiss/20201226_v2/',
    val_ratio=2/10,
    n_jobs=8)

sessions/nagiss/20201226_v2/ ['sessions/nagiss/20201226_v2/7994638.json', 'sessions/nagiss/20201226_v2/7991093.json', 'sessions/nagiss/20201226_v2/7988820.json', 'sessions/nagiss/20201226_v2/8386716.json', 'sessions/nagiss/20201226_v2/8038792.json', 'sessions/nagiss/20201226_v2/8233786.json', 'sessions/nagiss/20201226_v2/8073493.json', 'sessions/nagiss/20201226_v2/8012231.json', 'sessions/nagiss/20201226_v2/7995268.json', 'sessions/nagiss/20201226_v2/8046220.json', 'sessions/nagiss/20201226_v2/8321309.json', 'sessions/nagiss/20201226_v2/8102725.json', 'sessions/nagiss/20201226_v2/8146617.json', 'sessions/nagiss/20201226_v2/7986639.json', 'sessions/nagiss/20201226_v2/8029737.json', 'sessions/nagiss/20201226_v2/8089900.json', 'sessions/nagiss/20201226_v2/8354985.json', 'sessions/nagiss/20201226_v2/7995644.json', 'sessions/nagiss/20201226_v2/7998867.json', 'sessions/nagiss/20201226_v2/8257767.json', 'sessions/nagiss/20201226_v2/7992085.json', 'sessions/nagiss/20201226_v2/7987842.json', 's

  0%|          | 0/60 [00:00<?, ?it/s]

error while parse session sessions/nagiss/20201226_v2/.ipynb_checkpoints/8002300-checkpoint.json : 


100%|██████████| 14/14 [00:00<00:00, 1799.52it/s]


In [10]:
# datasets_nagiss_big[0].save_dataset('datasets/all_actions_nagiss_v2_train_top_10')
# datasets_nagiss_big[1].save_dataset('datasets/all_actions_nagiss_v2_val_top_10')

In [11]:
train,val = datasets_nagiss_big[0],datasets_nagiss_big[1]

In [9]:
def fix_shapes(train,val,H=72):
    train.X = train.X.reshape(-1, 100, H)
    val.X = val.X.reshape(-1, 100, H)

    train.actions = train.actions.reshape(-1)
    val.actions = val.actions.reshape(-1)

In [10]:
class NNWithCusomFeatures2(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        INPUT_F_C = INPUT_F + 2 * INPUT_F
        self.model_ff =  nn.Sequential(
            nn.BatchNorm1d(INPUT_F_C),
            nn.Dropout(DROP_P),
            nn.Linear(INPUT_F_C, H),
            nn.Sigmoid(),
            nn.Linear(H, H),
            nn.Sigmoid(),
            nn.Dropout(DROP_P),
            nn.Linear(H, 1)
#             nn.Sigmoid()
        )

    def forward(self, x):
        x = x[:, :36]   # v9
        lg = torch.log(1 + torch.abs(x))
        sn = torch.sin(x)
        input_x = torch.cat([x, lg, sn], axis=1)
        return self.model_ff(input_x)

In [30]:
INPUT_F = 72
H = 256
DROP_P = 0.01

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.sum(policy * rewards_batch) / torch.sum(rewards_batch)
    return out

learn(train, val, model, freq=50, batch_size=256,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 50 batch: 4.14671
val loss in 1 epoch: 4.21345
train loss in 1 epoch in 100 batch: 3.45924
val loss in 1 epoch: 3.56803
train loss in 1 epoch in 150 batch: 3.17914
val loss in 1 epoch: 3.18435
train loss in 1 epoch in 200 batch: 2.83735
val loss in 1 epoch: 3.09126
train loss in 1 epoch in 250 batch: 2.93774
val loss in 1 epoch: 3.05854
train loss in 1 epoch in 300 batch: 3.13401
val loss in 1 epoch: 3.03785
train loss in 1 epoch in 350 batch: 2.78217
val loss in 1 epoch: 3.02176
train loss in 1 epoch in 400 batch: 2.97525
val loss in 1 epoch: 3.00114
train loss in 1 epoch in 450 batch: 3.14768
val loss in 1 epoch: 2.99071
train loss in 2 epoch in 50 batch: 2.68735
val loss in 2 epoch: 2.95458
train loss in 2 epoch in 100 batch: 2.76762
val loss in 2 epoch: 2.94053
train loss in 2 epoch in 150 batch: 3.29673
val loss in 2 epoch: 2.92879
train loss in 2 epoch in 200 batch: 2.46205
val loss in 2 epoch: 2.92022
train loss in 2 epoch in 250 batch: 2.95611
val loss 

In [31]:
torch.save(model.state_dict(), 'models/nagiss_v8')

In [35]:
# v8
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

634.0 678.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
612.0 638.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
752.0 737.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
577.0 592.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
612.0 599.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
528.0 610.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
578.0 599.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
617.0 677.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
591.0 628.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
618.0 722.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py


(0.0016427853489932535,
 -36.1,
 36.261411996776964,
 0.2,
 'tmp/b_0.21800793341522473.py',
 'tmp/b_0.36094695632683116.py',
 array([634., 612., 752., 577., 612., 528., 578., 617., 591., 618.]),
 array([678., 638., 737., 592., 599., 610., 599., 677., 628., 722.]))

In [37]:
# v8
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

656.0 670.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
574.0 584.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
637.0 617.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
644.0 635.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
583.0 608.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
638.0 630.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
662.0 635.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
588.0 620.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
595.0 629.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py
656.0 678.0 tmp/b_0.8789289661002517.py tmp/b_0.4196354505275198.py


(0.26705232859577455,
 -7.3,
 20.7992788336519,
 0.4,
 'tmp/b_0.8789289661002517.py',
 'tmp/b_0.4196354505275198.py',
 array([656., 574., 637., 644., 583., 638., 662., 588., 595., 656.]),
 array([670., 584., 617., 635., 608., 630., 635., 620., 629., 678.]))

In [39]:
INPUT_F = 72
H = 256
DROP_P = 0.01

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(train, val, model, freq=50, batch_size=256,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 50 batch: 1.26886
val loss in 1 epoch: 1.30705
train loss in 1 epoch in 100 batch: 1.15985
val loss in 1 epoch: 1.05718
train loss in 1 epoch in 150 batch: 1.07963
val loss in 1 epoch: 0.92873
train loss in 1 epoch in 200 batch: 0.93143
val loss in 1 epoch: 0.89737
train loss in 1 epoch in 250 batch: 0.93955
val loss in 1 epoch: 0.88664
train loss in 1 epoch in 300 batch: 1.15136
val loss in 1 epoch: 0.88071
train loss in 1 epoch in 350 batch: 0.86700
val loss in 1 epoch: 0.87526
train loss in 1 epoch in 400 batch: 0.97938
val loss in 1 epoch: 0.86932
train loss in 1 epoch in 450 batch: 0.96049
val loss in 1 epoch: 0.86571
train loss in 2 epoch in 50 batch: 0.82117
val loss in 2 epoch: 0.85503
train loss in 2 epoch in 100 batch: 0.74852
val loss in 2 epoch: 0.85012
train loss in 2 epoch in 150 batch: 1.01651
val loss in 2 epoch: 0.84566
train loss in 2 epoch in 200 batch: 0.74224
val loss in 2 epoch: 0.84266
train loss in 2 epoch in 250 batch: 0.97031
val loss 

In [40]:
torch.save(model.state_dict(), 'models/nagiss_v9')

In [42]:
# v9
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

588.0 598.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
635.0 648.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
647.0 671.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
614.0 635.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
642.0 613.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
603.0 609.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
528.0 564.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
675.0 680.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
641.0 660.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
634.0 658.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py


(0.014015995659641567,
 -12.9,
 16.603915200939806,
 0.1,
 'tmp/b_0.21800793341522473.py',
 'tmp/b_0.36094695632683116.py',
 array([588., 635., 647., 614., 642., 603., 528., 675., 641., 634.]),
 array([598., 648., 671., 635., 613., 609., 564., 680., 660., 658.]))

In [70]:
# v11
INPUT_F = 36
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(train, val, model, freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 2.85484
val loss in 1 epoch: 3.56275
train loss in 1 epoch in 400 batch: 3.92476
val loss in 1 epoch: 3.28502
train loss in 1 epoch in 600 batch: 3.79547
val loss in 1 epoch: 3.25171
train loss in 1 epoch in 800 batch: 3.71395
val loss in 1 epoch: 3.23124
train loss in 1 epoch in 1000 batch: 2.36579
val loss in 1 epoch: 3.21198
train loss in 1 epoch in 1200 batch: 2.46426
val loss in 1 epoch: 3.19612
train loss in 1 epoch in 1400 batch: 5.10417
val loss in 1 epoch: 3.17131
train loss in 1 epoch in 1600 batch: 1.91599
val loss in 1 epoch: 3.17259
train loss in 1 epoch in 1800 batch: 2.31170
val loss in 1 epoch: 3.13965
train loss in 1 epoch in 2000 batch: 2.79342
val loss in 1 epoch: 3.12516
train loss in 1 epoch in 2200 batch: 3.22354
val loss in 1 epoch: 3.12646
train loss in 1 epoch in 2400 batch: 3.18674
val loss in 1 epoch: 3.09653
train loss in 1 epoch in 2600 batch: 2.67877
val loss in 1 epoch: 3.10167
train loss in 1 epoch in 2800 batch: 2.626

KeyboardInterrupt: 

In [54]:
torch.save(model.state_dict(), 'models/nagiss_v11')

In [72]:
# v11
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

629.0 627.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
626.0 585.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
673.0 614.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
673.0 674.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
648.0 610.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
678.0 651.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
668.0 655.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
629.0 637.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
622.0 590.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py
635.0 621.0 tmp/b_0.5686394484909753.py tmp/b_0.24554733352265246.py


(0.0006848808459747456,
 21.7,
 20.209156340629363,
 0.8,
 'tmp/b_0.5686394484909753.py',
 'tmp/b_0.24554733352265246.py',
 array([629., 626., 673., 673., 648., 678., 668., 629., 622., 635.]),
 array([627., 585., 614., 674., 610., 651., 655., 637., 590., 621.]))

In [74]:
# v11 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

635.0 669.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
672.0 674.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
698.0 630.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
638.0 571.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
585.0 607.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
697.0 645.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
628.0 640.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
612.0 559.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
616.0 644.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py
654.0 577.0 tmp/b_0.9458669928299895.py tmp/b_0.7455212560064984.py


(0.10576402451141403,
 21.9,
 42.81460031344448,
 0.5,
 'tmp/b_0.9458669928299895.py',
 'tmp/b_0.7455212560064984.py',
 array([635., 672., 698., 638., 585., 697., 628., 612., 616., 654.]),
 array([669., 674., 630., 571., 607., 645., 640., 559., 644., 577.]))

In [101]:
#analyze v11
model.load_state_dict(torch.load("models/nagiss_v11"))
model.eval()

model.state_dict().keys()

np.argsort(np.abs(np.mean(model.state_dict()['model_ff.2.weight'].numpy(),axis=0)))

array([ 57,  66,  97,  75,  10,  42,  34,  86, 101,  93,  22,   6,  46,
        33,  81, 106,  11, 105,  72,  60, 103,  27,  73,  82,  69,  94,
        32,  18,   1,  47,  89,  90,  70,  95,  61,  99,  74,  20,  63,
        80,  53,  76,  38,  77,  13,  28,  71,  36,  59,  49,  24,  92,
        41,  25,  54, 100,  31,  62,  64,  84,  16,  30,  35, 107,  23,
        50,  78,  98,  12,  56,  65,  68,  26,  17,  44, 104,  96,  85,
         0,   8,  37,  79,  14,  40, 102,  48,  21,   2,   9,  58,  29,
         5,  83,  91,  43,  88,  19,   4,  52,  67,  15,  51,   3,  87,
        55,  39,  45,   7])

In [102]:
# v12
INPUT_F = 72
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(train, val, model, freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 0.49264
val loss in 1 epoch: 0.97389
train loss in 1 epoch in 400 batch: 0.25059
val loss in 1 epoch: 0.90190
train loss in 1 epoch in 600 batch: 1.16856
val loss in 1 epoch: 0.88345
train loss in 1 epoch in 800 batch: 1.58572
val loss in 1 epoch: 0.87793
train loss in 1 epoch in 1000 batch: 0.72290
val loss in 1 epoch: 0.86430
train loss in 1 epoch in 1200 batch: 1.10172
val loss in 1 epoch: 0.86087
train loss in 1 epoch in 1400 batch: 0.61443
val loss in 1 epoch: 0.85290
train loss in 1 epoch in 1600 batch: 0.70160
val loss in 1 epoch: 0.85688
train loss in 1 epoch in 1800 batch: 0.77548
val loss in 1 epoch: 0.84472
train loss in 1 epoch in 2000 batch: 0.36067
val loss in 1 epoch: 0.84141
train loss in 1 epoch in 2200 batch: 1.35711
val loss in 1 epoch: 0.84295
train loss in 1 epoch in 2400 batch: 0.79576
val loss in 1 epoch: 0.83438
train loss in 1 epoch in 2600 batch: 0.80853
val loss in 1 epoch: 0.84309
train loss in 1 epoch in 2800 batch: 0.646

In [103]:
torch.save(model.state_dict(), 'models/nagiss_v12')

In [105]:
# v12
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

687.0 667.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
714.0 712.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
651.0 634.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
621.0 603.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
681.0 637.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
637.0 614.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
611.0 613.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
629.0 577.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
629.0 606.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
647.0 624.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py


(7.3509443816825105e-06,
 22.0,
 15.517731793016658,
 0.9,
 'tmp/b_0.21800793341522473.py',
 'tmp/b_0.36094695632683116.py',
 array([687., 714., 651., 621., 681., 637., 611., 629., 629., 647.]),
 array([667., 712., 634., 603., 637., 614., 613., 577., 606., 624.]))

In [106]:
# v12 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

631.0 658.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
651.0 616.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
613.0 623.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
701.0 644.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
682.0 652.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
643.0 670.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
656.0 574.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
670.0 630.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
626.0 666.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py
700.0 678.0 tmp/b_0.8036731827992181.py tmp/b_0.6096653281782922.py


(0.18173072385616584,
 16.2,
 38.360917611548345,
 0.6,
 'tmp/b_0.8036731827992181.py',
 'tmp/b_0.6096653281782922.py',
 array([631., 651., 613., 701., 682., 643., 656., 670., 626., 700.]),
 array([658., 616., 623., 644., 652., 670., 574., 630., 666., 678.]))

In [113]:
torch.Tensor(np.arange(12)).reshape(3,4), torch.Tensor(np.arange(12)).reshape(3,4)[[0,1,2],[0,1,1]]

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([0., 5., 9.]))

In [137]:
# v13
INPUT_F = 72
H = 256
DROP_P = 0.1

baseline_f = 9  # gittins feature

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient_with_baseline(logits_batch, rewards_batch, actions_batch, X_batch):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    
#     baselines = X_batch[:, baseline_f].detach()
#     baselines = torch.reshape(baselines, (-1, 100))
#     baselines_needed = baselines[np.arange(baselines.shape[0]), actions_batch.detach().numpy()]

    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * (rewards_batch - 0.5))
    return out

learn(train, val, model, freq=200,
      batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient_with_baseline)

train loss in 1 epoch in 200 batch: -1.62668
val loss in 1 epoch: -0.91307
train loss in 1 epoch in 400 batch: -6.53250
val loss in 1 epoch: -2.57788
train loss in 1 epoch in 600 batch: -5.15921
val loss in 1 epoch: -4.05404
train loss in 1 epoch in 800 batch: -4.44009
val loss in 1 epoch: -5.12954
train loss in 1 epoch in 1000 batch: -7.12367
val loss in 1 epoch: -6.16595
train loss in 1 epoch in 1200 batch: -4.96797
val loss in 1 epoch: -7.14651
train loss in 1 epoch in 1400 batch: -13.49207
val loss in 1 epoch: -8.11245
train loss in 1 epoch in 1600 batch: -7.55415
val loss in 1 epoch: -9.07644
train loss in 1 epoch in 1800 batch: -10.27917
val loss in 1 epoch: -9.91737
train loss in 1 epoch in 2000 batch: -22.79364
val loss in 1 epoch: -10.84522
train loss in 1 epoch in 2200 batch: -3.48836
val loss in 1 epoch: -11.78287
train loss in 1 epoch in 2400 batch: -14.74745
val loss in 1 epoch: -12.66552
train loss in 1 epoch in 2600 batch: -7.11691
val loss in 1 epoch: -13.51396
train lo

In [138]:
torch.save(model.state_dict(), 'models/nagiss_v13')

In [139]:
# v13
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

509.0 597.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
532.0 696.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
547.0 636.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
526.0 646.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
501.0 640.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py
504.0 638.0 tmp/b_0.21800793341522473.py tmp/b_0.36094695632683116.py


KeyboardInterrupt: 

In [123]:
model.load_state_dict(torch.load("models/nagiss_v13"))

<All keys matched successfully>

In [127]:
model.state_dict()['model_ff.2.weight']

tensor([[ 0.0269,  0.0722,  0.0958,  ...,  0.0060, -0.0049, -0.0259],
        [-0.0678, -0.1197, -0.1033,  ...,  0.0255, -0.0558, -0.0673],
        [ 0.0392,  0.1247,  0.0980,  ...,  0.0802, -0.0334,  0.0140],
        ...,
        [ 0.0385,  0.0472,  0.1031,  ...,  0.0349,  0.0768,  0.0432],
        [ 0.0998,  0.0202,  0.1445,  ..., -0.0355, -0.0189,  0.0373],
        [ 0.1110,  0.0506,  0.0584,  ...,  0.0982,  0.0425,  0.0193]])

In [130]:
# v14
INPUT_F = 72
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_copy(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy)
    return out

learn(train, val, model, freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_copy)

train loss in 1 epoch in 200 batch: 2.92199
val loss in 1 epoch: 3.35883
train loss in 1 epoch in 400 batch: 3.02275
val loss in 1 epoch: 3.25613
train loss in 1 epoch in 600 batch: 3.82501
val loss in 1 epoch: 3.20623
train loss in 1 epoch in 800 batch: 3.61449
val loss in 1 epoch: 3.17061
train loss in 1 epoch in 1000 batch: 3.38468
val loss in 1 epoch: 3.13840
train loss in 1 epoch in 1200 batch: 3.35493
val loss in 1 epoch: 3.11387
train loss in 1 epoch in 1400 batch: 3.62872
val loss in 1 epoch: 3.11241
train loss in 1 epoch in 1600 batch: 2.70347
val loss in 1 epoch: 3.13029
train loss in 1 epoch in 1800 batch: 2.82388
val loss in 1 epoch: 3.10540
train loss in 1 epoch in 2000 batch: 2.82319
val loss in 1 epoch: 3.10048
train loss in 1 epoch in 2200 batch: 3.38223
val loss in 1 epoch: 3.09638
train loss in 1 epoch in 2400 batch: 3.38437
val loss in 1 epoch: 3.07828
train loss in 1 epoch in 2600 batch: 2.98415
val loss in 1 epoch: 3.10255
train loss in 1 epoch in 2800 batch: 2.592

In [132]:
torch.save(model.state_dict(), 'models/nagiss_v14')

In [141]:
# v14
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

702.0 652.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
566.0 586.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
596.0 582.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
692.0 627.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
626.0 584.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
663.0 648.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
690.0 592.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
687.0 702.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
678.0 649.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py
576.0 524.0 tmp/b_0.9104238836280365.py tmp/b_0.5616156202153793.py


(0.0023762587560710747,
 33.0,
 34.34239362653687,
 0.8,
 'tmp/b_0.9104238836280365.py',
 'tmp/b_0.5616156202153793.py',
 array([702., 566., 596., 692., 626., 663., 690., 687., 678., 576.]),
 array([652., 586., 582., 627., 584., 648., 592., 702., 649., 524.]))

In [142]:
# v14 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,True", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

566.0 588.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
677.0 644.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
658.0 628.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
611.0 599.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
643.0 572.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
612.0 609.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
613.0 641.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
641.0 665.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
634.0 657.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py
669.0 627.0 tmp/b_0.26055979727588763.py tmp/b_0.955884903168955.py


(0.3575011703035106,
 9.4,
 32.30541750233233,
 0.6,
 'tmp/b_0.26055979727588763.py',
 'tmp/b_0.955884903168955.py',
 array([566., 677., 658., 611., 643., 612., 613., 641., 634., 669.]),
 array([588., 644., 628., 599., 572., 609., 641., 665., 657., 627.]))

### nagiss in top 10

In [11]:
nagiss_top_10 = snowden.collect_dataset_from_dir(
    neural_agent_class,
    'sessions/20201227/18775092',
    val_ratio=2/10,
    n_jobs=8)

sessions/20201227/18775092 ['sessions/20201227/18775092/8667645.json', 'sessions/20201227/18775092/9895519.json', 'sessions/20201227/18775092/7994638.json', 'sessions/20201227/18775092/7991093.json', 'sessions/20201227/18775092/7988820.json', 'sessions/20201227/18775092/9818142.json', 'sessions/20201227/18775092/8718088.json', 'sessions/20201227/18775092/8386716.json', 'sessions/20201227/18775092/9782190.json', 'sessions/20201227/18775092/8038792.json', 'sessions/20201227/18775092/9748485.json', 'sessions/20201227/18775092/8652496.json', 'sessions/20201227/18775092/9800050.json', 'sessions/20201227/18775092/8233786.json', 'sessions/20201227/18775092/8073493.json', 'sessions/20201227/18775092/9042670.json', 'sessions/20201227/18775092/8012231.json', 'sessions/20201227/18775092/9609526.json', 'sessions/20201227/18775092/8752347.json', 'sessions/20201227/18775092/9345093.json', 'sessions/20201227/18775092/7995268.json', 'sessions/20201227/18775092/8046220.json', 'sessions/20201227/1877509

100%|██████████| 32/32 [01:16<00:00,  2.40s/it]


In [12]:
fix_dataset(nagiss_top_10[0], nagiss_top_10[1], 36)

In [164]:
# v15
INPUT_F = 36
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(nagiss_top_10[0], nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 1.36220
val loss in 1 epoch: 0.99618
train loss in 1 epoch in 400 batch: 0.31884
val loss in 1 epoch: 0.91305
train loss in 1 epoch in 600 batch: 0.66208
val loss in 1 epoch: 0.90011
train loss in 1 epoch in 800 batch: 0.61705
val loss in 1 epoch: 0.89267
train loss in 1 epoch in 1000 batch: 0.06988
val loss in 1 epoch: 0.88315
train loss in 1 epoch in 1200 batch: 0.65496
val loss in 1 epoch: 0.87599
train loss in 1 epoch in 1400 batch: 1.08106
val loss in 1 epoch: 0.86881
train loss in 1 epoch in 1600 batch: 0.11978
val loss in 1 epoch: 0.86409
train loss in 1 epoch in 1800 batch: 0.29621
val loss in 1 epoch: 0.85758
train loss in 1 epoch in 2000 batch: 0.49103
val loss in 1 epoch: 0.85109
train loss in 1 epoch in 2200 batch: 0.99746
val loss in 1 epoch: 0.84623
train loss in 1 epoch in 2400 batch: 0.33409
val loss in 1 epoch: 0.84369
train loss in 1 epoch in 2600 batch: 1.36439
val loss in 1 epoch: 0.84170
train loss in 1 epoch in 2800 batch: 1.642

In [165]:
torch.save(model.state_dict(), 'models/nagiss_v15')

In [169]:
# v15
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

622.0 591.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
664.0 606.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
610.0 571.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
632.0 631.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
652.0 651.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
660.0 562.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
607.0 644.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
679.0 660.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
666.0 645.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
653.0 625.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py


(0.016639938795987075,
 25.9,
 34.20365477547684,
 0.9,
 'tmp/b_0.8650894478131601.py',
 'tmp/b_0.8547409200351491.py',
 array([622., 664., 610., 632., 652., 660., 607., 679., 666., 653.]),
 array([591., 606., 571., 631., 651., 562., 644., 660., 645., 625.]))

In [170]:
# v15 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

630.0 594.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
625.0 642.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
699.0 667.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
643.0 643.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
674.0 610.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
723.0 737.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
586.0 654.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
633.0 657.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
651.0 595.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py
644.0 655.0 tmp/b_0.41885852893729714.py tmp/b_0.11874647569664065.py


(0.6599838197678104,
 5.4,
 38.81546083714581,
 0.5,
 'tmp/b_0.41885852893729714.py',
 'tmp/b_0.11874647569664065.py',
 array([630., 625., 699., 643., 674., 723., 586., 633., 651., 644.]),
 array([594., 642., 667., 643., 610., 737., 654., 657., 595., 655.]))

In [13]:
class Attention(nn.Module):
    """ Applies attention mechanism on the `context` using the `query`.

    **Thank you** to IBM for their initial implementation of :class:`Attention`. Here is
    their `License
    <https://github.com/IBM/pytorch-seq2seq/blob/master/LICENSE>`__.

    Args:
        dimensions (int): Dimensionality of the query and context.
        attention_type (str, optional): How to compute the attention score:

            * dot: :math:`score(H_j,q) = H_j^T q`
            * general: :math:`score(H_j, q) = H_j^T W_a q`

    Example:

         >>> attention = Attention(256)
         >>> query = torch.randn(5, 1, 256)
         >>> context = torch.randn(5, 5, 256)
         >>> output, weights = attention(query, context)
         >>> output.size()
         torch.Size([5, 1, 256])
         >>> weights.size()
         torch.Size([5, 1, 5])
    """

    def __init__(self, dimensions, attention_type='general'):
        super(Attention, self).__init__()

        if attention_type not in ['dot', 'general']:
            raise ValueError('Invalid attention type selected.')

        self.attention_type = attention_type
        if self.attention_type == 'general':
            self.linear_in = nn.Linear(dimensions, dimensions, bias=False)

        self.linear_out = nn.Linear(dimensions * 2, dimensions, bias=False)
        self.softmax = nn.Softmax(dim=-1)
        self.tanh = nn.Tanh()

    def forward(self, query, context):
        """
        Args:
            query (:class:`torch.FloatTensor` [batch size, output length, dimensions]): Sequence of
                queries to query the context.
            context (:class:`torch.FloatTensor` [batch size, query length, dimensions]): Data
                overwhich to apply the attention mechanism.

        Returns:
            :class:`tuple` with `output` and `weights`:
            * **output** (:class:`torch.LongTensor` [batch size, output length, dimensions]):
              Tensor containing the attended features.
            * **weights** (:class:`torch.FloatTensor` [batch size, output length, query length]):
              Tensor containing attention weights.
        """
        batch_size, output_len, dimensions = query.size()
        query_len = context.size(1)

        if self.attention_type == "general":
            query = query.reshape(batch_size * output_len, dimensions)
            query = self.linear_in(query)
            query = query.reshape(batch_size, output_len, dimensions)

        # TODO: Include mask on PADDING_INDEX?

        # (batch_size, output_len, dimensions) * (batch_size, query_len, dimensions) ->
        # (batch_size, output_len, query_len)
        attention_scores = torch.bmm(query, context.transpose(1, 2).contiguous())

        # Compute weights across every context sequence
        attention_scores = attention_scores.view(batch_size * output_len, query_len)
        attention_weights = self.softmax(attention_scores)
        attention_weights = attention_weights.view(batch_size, output_len, query_len)

        # (batch_size, output_len, query_len) * (batch_size, query_len, dimensions) ->
        # (batch_size, output_len, dimensions)
        mix = torch.bmm(attention_weights, context)

        # concat -> (batch_size * output_len, 2*dimensions)
        combined = torch.cat((mix, query), dim=2)
        combined = combined.view(batch_size * output_len, 2 * dimensions)

        # Apply linear_out on every 2nd dimension of concat
        # output -> (batch_size, output_len, dimensions)
        output = self.linear_out(combined).view(batch_size, output_len, dimensions)
        output = self.tanh(output)

        return output, attention_weights

In [14]:
class NNWithAttention(nn.Module):
    def __init__(self, INPUT_F, DROP_P, H):
        super().__init__()
        self.H = H
        self.O = 100
        INPUT_F_C = INPUT_F + 2 * INPUT_F
        self.model_ff_1 =  nn.Sequential(
            nn.BatchNorm1d(INPUT_F_C),
            nn.Dropout(DROP_P),
            nn.Linear(INPUT_F_C, H),
            nn.Sigmoid(),
            nn.Linear(H, H),
#             nn.Sigmoid(),
#             nn.Dropout(DROP_P),
#             nn.Linear(H, 1)
#             nn.Sigmoid()
        )
        
        self.attn = Attention(self.H, attention_type='general')
        
        self.model_ff_2 =  nn.Sequential(
            nn.Linear(H, H),
            nn.Sigmoid(),
            nn.Dropout(DROP_P),
            nn.Linear(H, 1),
#             nn.Sigmoid(),
#             nn.Dropout(DROP_P),
#             nn.Linear(H, 1)
#             nn.Sigmoid()
        )
        
    def make_features(self, x):
        x = x[:, :36]   # v9
        lg = torch.log(1 + torch.abs(x))
        sn = torch.sin(x)
        input_x = torch.cat([x, lg, sn], axis=1)
        return input_x

    def forward(self, x):
        x = self.make_features(x)
        query = self.model_ff_1(x)
        hidden = torch.reshape(query, (-1, self.O, self.H))
        B = hidden.shape[0]
        b_ones = torch.ones((self.O, self.O))
#         mask = torch.block_diag([b_ones for _ in np.arange(B)])
        mask = 1
#         print(query.shape, hidden.shape)
#         assert False
        weighted_query, weights = self.attn(hidden, hidden)
        weighted_query = torch.reshape(weighted_query, (-1, self.H))
        out = self.model_ff_2(weighted_query)
#         print(query.shape, hidden.shape, weighted_query.shape, weights.shape, out.shape)
        return out

In [11]:
# v16
INPUT_F = 36
H = 256
DROP_P = 0.1

model = NNWithAttention(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(nagiss_top_10[0], nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 1.11466
val loss in 1 epoch: 0.91816
train loss in 1 epoch in 400 batch: 0.37721
val loss in 1 epoch: 0.90005
train loss in 1 epoch in 600 batch: 0.63423
val loss in 1 epoch: 0.86460
train loss in 1 epoch in 800 batch: 0.53577
val loss in 1 epoch: 0.85394
train loss in 1 epoch in 1000 batch: 0.06103
val loss in 1 epoch: 0.87833
train loss in 1 epoch in 1200 batch: 0.60838
val loss in 1 epoch: 0.87533
train loss in 1 epoch in 1400 batch: 1.06349
val loss in 1 epoch: 0.84282
train loss in 1 epoch in 1600 batch: 0.15884
val loss in 1 epoch: 0.84613
train loss in 1 epoch in 1800 batch: 0.26516
val loss in 1 epoch: 0.83164
train loss in 1 epoch in 2000 batch: 0.40338
val loss in 1 epoch: 0.83390
train loss in 1 epoch in 2200 batch: 1.01126
val loss in 1 epoch: 0.83487
train loss in 1 epoch in 2400 batch: 0.24559
val loss in 1 epoch: 0.82700
train loss in 1 epoch in 2600 batch: 1.43094
val loss in 1 epoch: 0.83151
train loss in 1 epoch in 2800 batch: 1.658

In [12]:
torch.save(model.state_dict(), 'models/nagiss_v16')

In [13]:
2 + 2

4

In [16]:
# v16
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

598.0 585.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
593.0 613.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
648.0 609.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
621.0 557.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
580.0 569.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
639.0 696.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
625.0 621.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
654.0 687.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
638.0 623.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
659.0 610.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py


(0.4482659057750409,
 8.5,
 35.44643846707311,
 0.7,
 'tmp/b_0.8650894478131601.py',
 'tmp/b_0.8547409200351491.py',
 array([598., 593., 648., 621., 580., 639., 625., 654., 638., 659.]),
 array([585., 613., 609., 557., 569., 696., 621., 687., 623., 610.]))

In [17]:
# v16 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

621.0 614.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
621.0 631.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
621.0 646.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
647.0 697.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
599.0 585.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
599.0 626.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
666.0 671.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
639.0 616.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
629.0 609.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py
611.0 590.0 tmp/b_0.7746478479790496.py tmp/b_0.1629299312838185.py


(0.6658738488700195,
 -3.2,
 23.434163095788165,
 0.5,
 'tmp/b_0.7746478479790496.py',
 'tmp/b_0.1629299312838185.py',
 array([621., 621., 621., 647., 599., 599., 666., 639., 629., 611.]),
 array([614., 631., 646., 697., 585., 626., 671., 616., 609., 590.]))

In [18]:
# v17
INPUT_F = 36
H = 256
DROP_P = 0.5

model = NNWithAttention(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(nagiss_top_10[0], nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=2, criterion=policy_gradient)  # stoped manualy

train loss in 1 epoch in 200 batch: 1.37473
val loss in 1 epoch: 0.95067
train loss in 1 epoch in 400 batch: 0.39388
val loss in 1 epoch: 0.91126
train loss in 1 epoch in 600 batch: 0.63603
val loss in 1 epoch: 0.88273
train loss in 1 epoch in 800 batch: 0.60204
val loss in 1 epoch: 0.87285
train loss in 1 epoch in 1000 batch: 0.09691
val loss in 1 epoch: 0.87556
train loss in 1 epoch in 1200 batch: 0.64043
val loss in 1 epoch: 0.85667
train loss in 1 epoch in 1400 batch: 1.10523
val loss in 1 epoch: 0.84905
train loss in 1 epoch in 1600 batch: 0.17899
val loss in 1 epoch: 0.84598
train loss in 1 epoch in 1800 batch: 0.29390
val loss in 1 epoch: 0.84049
train loss in 1 epoch in 2000 batch: 0.40744
val loss in 1 epoch: 0.84137
train loss in 1 epoch in 2200 batch: 1.00212
val loss in 1 epoch: 0.83570
train loss in 1 epoch in 2400 batch: 0.32799
val loss in 1 epoch: 0.83877
train loss in 1 epoch in 2600 batch: 1.40992
val loss in 1 epoch: 0.83460
train loss in 1 epoch in 2800 batch: 1.729

KeyboardInterrupt: 

In [19]:
torch.save(model.state_dict(), 'models/nagiss_v17')

In [21]:
# v17
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

672.0 635.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
666.0 633.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
698.0 668.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
679.0 653.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
681.0 648.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
645.0 627.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
733.0 699.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
601.0 605.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
623.0 638.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py
658.0 616.0 tmp/b_0.9715554847378113.py tmp/b_0.7203789564181606.py


(2.9026572458509727e-05,
 23.4,
 17.69858751426226,
 0.8,
 'tmp/b_0.9715554847378113.py',
 'tmp/b_0.7203789564181606.py',
 array([672., 666., 698., 679., 681., 645., 733., 601., 623., 658.]),
 array([635., 633., 668., 653., 648., 627., 699., 605., 638., 616.]))

In [22]:
# v17 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

737.0 637.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
625.0 579.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
592.0 616.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
598.0 562.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
651.0 640.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
580.0 582.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
603.0 601.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
642.0 593.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
673.0 686.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py
591.0 582.0 tmp/b_0.7786155387529811.py tmp/b_0.9054717873967233.py


(0.05257606029925348,
 21.4,
 34.911889092399456,
 0.7,
 'tmp/b_0.7786155387529811.py',
 'tmp/b_0.9054717873967233.py',
 array([737., 625., 592., 598., 651., 580., 603., 642., 673., 591.]),
 array([637., 579., 616., 562., 640., 582., 601., 593., 686., 582.]))

In [23]:
# v18
INPUT_F = 36
H = 256
DROP_P = 0.5

model = NNWithAttention(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(nagiss_top_10[0], nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)  # stoped manualy

train loss in 1 epoch in 200 batch: 1.18956
val loss in 1 epoch: 0.95670
train loss in 1 epoch in 400 batch: 0.37571
val loss in 1 epoch: 0.90069
train loss in 1 epoch in 600 batch: 0.63046
val loss in 1 epoch: 0.89248
train loss in 1 epoch in 800 batch: 0.56747
val loss in 1 epoch: 0.87396
train loss in 1 epoch in 1000 batch: 0.05146
val loss in 1 epoch: 0.87221
train loss in 1 epoch in 1200 batch: 0.64635
val loss in 1 epoch: 0.85244
train loss in 1 epoch in 1400 batch: 1.05073
val loss in 1 epoch: 0.84647
train loss in 1 epoch in 1600 batch: 0.14167
val loss in 1 epoch: 0.84404
train loss in 1 epoch in 1800 batch: 0.27423
val loss in 1 epoch: 0.83871
train loss in 1 epoch in 2000 batch: 0.42207
val loss in 1 epoch: 0.83573
train loss in 1 epoch in 2200 batch: 1.13866
val loss in 1 epoch: 0.83807
train loss in 1 epoch in 2400 batch: 0.34878
val loss in 1 epoch: 0.83783
train loss in 1 epoch in 2600 batch: 1.37678
val loss in 1 epoch: 0.84028
train loss in 1 epoch in 2800 batch: 1.486

In [24]:
torch.save(model.state_dict(), 'models/nagiss_v18')

In [25]:
# v18
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

566.0 555.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
600.0 616.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
567.0 574.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
638.0 626.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
566.0 544.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
629.0 556.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
620.0 624.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
631.0 556.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
666.0 650.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py
599.0 606.0 tmp/b_0.8650894478131601.py tmp/b_0.8547409200351491.py


(0.06907289544872972,
 17.5,
 30.44092639851816,
 0.6,
 'tmp/b_0.8650894478131601.py',
 'tmp/b_0.8547409200351491.py',
 array([566., 600., 567., 638., 566., 629., 620., 631., 666., 599.]),
 array([555., 616., 574., 626., 544., 556., 624., 556., 650., 606.]))

In [31]:
# v19
INPUT_F = 36
H = 256
DROP_P = 0.5

model = NNWithAttention(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    probs = torch.nn.Softmax(dim=1)(logits_batch)
    entropy = -torch.mean(probs * torch.log(probs))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch) + entropy
    return out

learn(nagiss_top_10[0], nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 1.07600
val loss in 1 epoch: 0.97636
train loss in 1 epoch in 400 batch: 0.40775
val loss in 1 epoch: 0.92396
train loss in 1 epoch in 600 batch: 0.63201
val loss in 1 epoch: 0.90262
train loss in 1 epoch in 800 batch: 0.52119
val loss in 1 epoch: 0.89054
train loss in 1 epoch in 1000 batch: 0.08349
val loss in 1 epoch: 0.89122
train loss in 1 epoch in 1200 batch: 0.71327
val loss in 1 epoch: 0.88878
train loss in 1 epoch in 1400 batch: 1.11178
val loss in 1 epoch: 0.88482
train loss in 1 epoch in 1600 batch: 0.14835
val loss in 1 epoch: 0.87769
train loss in 1 epoch in 1800 batch: 0.31868
val loss in 1 epoch: 0.86475
train loss in 1 epoch in 2000 batch: 0.43355
val loss in 1 epoch: 0.86681
train loss in 1 epoch in 2200 batch: 1.10838
val loss in 1 epoch: 0.87277
train loss in 1 epoch in 2400 batch: 0.39577
val loss in 1 epoch: 0.86857
train loss in 1 epoch in 2600 batch: 1.37946
val loss in 1 epoch: 0.86005
train loss in 1 epoch in 2800 batch: 1.475

In [32]:
torch.save(model.state_dict(), 'models/nagiss_v19')

In [35]:
# v19
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

662.0 665.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
573.0 581.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
632.0 635.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
635.0 648.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
612.0 584.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
577.0 600.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
679.0 623.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
655.0 630.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
579.0 649.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py
636.0 622.0 tmp/b_0.5231521158081155.py tmp/b_0.6374564363717186.py


(0.9765543186290638,
 0.3,
 32.28017967731902,
 0.4,
 'tmp/b_0.5231521158081155.py',
 'tmp/b_0.6374564363717186.py',
 array([662., 573., 632., 635., 612., 577., 679., 655., 579., 636.]),
 array([665., 581., 635., 648., 584., 600., 623., 630., 649., 622.]))

In [36]:
# v19 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

617.0 638.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
646.0 628.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
640.0 563.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
590.0 580.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
606.0 576.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
677.0 633.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
642.0 576.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
652.0 608.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
592.0 544.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py
660.0 661.0 tmp/b_0.06293206958092812.py tmp/b_0.7981795034322536.py


(0.0005469605380743128,
 31.5,
 28.817529387509957,
 0.8,
 'tmp/b_0.06293206958092812.py',
 'tmp/b_0.7981795034322536.py',
 array([617., 646., 640., 590., 606., 677., 642., 652., 592., 660.]),
 array([638., 628., 563., 580., 576., 633., 576., 608., 544., 661.]))

### next data iteration

In [37]:
sub = 18836001  # best up to 20210107
utils.web.download_all_sessions_for_submission(sub, 'sessions/20210107/{}'.format(sub), delta=0.1)

In [15]:
nagiss_v2 = snowden.collect_dataset_from_dir(
    neural_agent_class,
    'sessions/20210107/18836001/',
    val_ratio=2/10,
    n_jobs=8)

sessions/20210107/18836001/ ['sessions/20210107/18836001/9709690.json', 'sessions/20210107/18836001/8531135.json', 'sessions/20210107/18836001/10067762.json', 'sessions/20210107/18836001/8528656.json', 'sessions/20210107/18836001/9521864.json', 'sessions/20210107/18836001/8526170.json', 'sessions/20210107/18836001/9434139.json', 'sessions/20210107/18836001/9886335.json', 'sessions/20210107/18836001/8918337.json', 'sessions/20210107/18836001/8537521.json', 'sessions/20210107/18836001/8533974.json', 'sessions/20210107/18836001/8792232.json', 'sessions/20210107/18836001/9354441.json', 'sessions/20210107/18836001/9975622.json', 'sessions/20210107/18836001/10031677.json', 'sessions/20210107/18836001/8529362.json', 'sessions/20210107/18836001/9321023.json', 'sessions/20210107/18836001/8839728.json', 'sessions/20210107/18836001/8533305.json', 'sessions/20210107/18836001/10049893.json', 'sessions/20210107/18836001/8603318.json', 'sessions/20210107/18836001/10163463.json', 'sessions/20210107/18

100%|██████████| 30/30 [00:51<00:00,  1.72s/it]


In [16]:
nagiss_top_10[0].X.shape, nagiss_v2[0].X.shape

((261869, 100, 36), (241879, 3600))

In [17]:
fix_dataset(nagiss_v2[0], nagiss_v2[1], 36)

In [18]:
data = utils.snowden.Dataset(None, None, None)

In [25]:
data.X = np.concatenate([nagiss_top_10[0].X, nagiss_v2[0].X], axis=0)
data.y = np.concatenate([nagiss_top_10[0].y, nagiss_v2[0].y], axis=0)
data.actions = np.concatenate([nagiss_top_10[0].actions, nagiss_v2[0].actions], axis=0)

In [26]:
data.X.shape

(503748, 100, 36)

In [27]:
# v22 - like v15 but x2 data
INPUT_F = 36
H = 256
DROP_P = 0.1

model = NNWithCusomFeatures2(INPUT_F, DROP_P, H)

def policy_gradient(logits_batch, rewards_batch, actions_batch, _):
    loss = nn.CrossEntropyLoss(reduction='none')
    logits_batch = torch.reshape(logits_batch, (-1, 100))
    policy = loss(logits_batch, actions_batch)
    out = torch.mean(policy * rewards_batch)
    return out

learn(data, nagiss_top_10[1], model,
      freq=200, batch_size=16,lr=1e-4, epochs=4, criterion=policy_gradient)

train loss in 1 epoch in 200 batch: 0.64893
val loss in 1 epoch: 1.01453
train loss in 1 epoch in 400 batch: 0.34390
val loss in 1 epoch: 0.91446
train loss in 1 epoch in 600 batch: 0.04847
val loss in 1 epoch: 0.90182
train loss in 1 epoch in 800 batch: 0.60163
val loss in 1 epoch: 0.89363
train loss in 1 epoch in 1000 batch: 0.27664
val loss in 1 epoch: 0.88565
train loss in 1 epoch in 1200 batch: 0.67074
val loss in 1 epoch: 0.87791
train loss in 1 epoch in 1400 batch: 0.85610
val loss in 1 epoch: 0.86868
train loss in 1 epoch in 1600 batch: 1.36075
val loss in 1 epoch: 0.86483
train loss in 1 epoch in 1800 batch: 1.58412
val loss in 1 epoch: 0.85623
train loss in 1 epoch in 2000 batch: 0.42443
val loss in 1 epoch: 0.85023
train loss in 1 epoch in 2200 batch: 0.86434
val loss in 1 epoch: 0.84689
train loss in 1 epoch in 2400 batch: 0.65696
val loss in 1 epoch: 0.84276
train loss in 1 epoch in 2600 batch: 0.37096
val loss in 1 epoch: 0.84024
train loss in 1 epoch in 2800 batch: 0.206

In [28]:
2 + 2

4

In [29]:
torch.save(model.state_dict(), 'models/nagiss_v22')

In [31]:
# v22
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=False", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

607.0 571.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
683.0 610.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
563.0 528.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
628.0 635.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
616.0 626.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
658.0 603.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
677.0 670.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
623.0 554.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
661.0 636.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py
706.0 669.0 tmp/b_0.11312095728679927.py tmp/b_0.9546663769656143.py


(0.00023920097700547846,
 32.0,
 27.546324618721822,
 0.8,
 'tmp/b_0.11312095728679927.py',
 'tmp/b_0.9546663769656143.py',
 array([607., 683., 563., 628., 616., 658., 677., 623., 661., 706.]),
 array([571., 610., 528., 635., 626., 603., 670., 554., 636., 669.]))

In [32]:
# v22 vs neural with mean
neural_with_nagiss = utils.bandits.neural.format("use_only_nagiss,use_mean=True,False", "")
neural_default = utils.bandits.neural.format("use_mean=True", "")
utils.bandits.compare(
    utils.bandits.Agent(text=neural_with_nagiss),
    utils.bandits.Agent(text=neural_default),
    verbose=True)

607.0 568.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
590.0 592.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
702.0 644.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
659.0 633.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
661.0 643.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
624.0 578.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
704.0 683.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
704.0 687.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
671.0 609.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py
676.0 664.0 tmp/b_0.41180196492681487.py tmp/b_0.2274189914325433.py


(1.989122719632633e-06,
 29.7,
 19.753733824267247,
 0.9,
 'tmp/b_0.41180196492681487.py',
 'tmp/b_0.2274189914325433.py',
 array([607., 590., 702., 659., 661., 624., 704., 704., 671., 676.]),
 array([568., 592., 644., 633., 643., 578., 683., 687., 609., 664.]))